In [ ]:
!pip install -r ../requirements.txt

In [1]:
import sys
sys.path.insert(0, "../")

# Downloading and processing data

There are two ways you can obtain OSM data in osm.pbf format:  
- Download it yourself: [parts of the world](https://download.geofabrik.de/), [cities](https://download.bbbike.org/osm/bbbike/), [adjustable area](https://extract.bbbike.org/) (via mail), [adjustable area](https://export.hotosm.org/en/v3/) (online), [planet](https://planet.maps.mail.ru/pbf/)
- Let the program download it for you

If the map is downloaded you can specify the filename:

In [ ]:
from visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
filename = "../maps/kozlovo.osm.pbf"
bbox = [36.2, 56.5, 36.7, 56.7]
vgraph.compute_geometry(bbox=bbox, filename=filename)

Or, alternatively, you can only specify the bounding box, and the map will be downloaded automatically ([curl](https://curl.se/) & [osmosis](https://wiki.openstreetmap.org/wiki/Osmosis) required):

In [ ]:
bbox = [34, 59, 34.2, 59.1]
vgraph.compute_geometry(bbox=bbox)

Data inside this area can be processed using VisibilityGraph with chosen or default parameters.  
If not specified, optimal parameters will be computed by the algorithm.

In [ ]:
vgraph.prune_geometry(epsilon_polygon=0.003,
                       epsilon_linestring=0.001,
                       bbox_comp=10)

Computed data can also be saved in .h5 file to skip data processing the next time:

In [ ]:
vgraph.save_geometry("../maps/user_area.h5")

# Using precomputed data and building visibility graph

In [2]:
from visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
vgraph.load_geometry("../maps/user_area.h5")

In [4]:
vgraph.polygons

,tag,geometry,convex_hull,convex_hull_points,angles
0,wetland,"(((34.1773601, 59.0005708), (34.1917453, 59.00...","((34.1354441, 59.0373975), (34.1272688, 59.030...","(6, 7, 11, 12, 14, 0, 1, 2, 3)","(3.816845040652781, 4.042617686739325, 4.08443..."
1,wetland,"(((34.0228613, 59.0155847), (34.0131933, 59.02...","((34.0020418, 59.0250026), (34.0096109, 59.003...","(7, 11, 12, 14, 3, 4, 6)","(5.04762648423328, 5.858518048190597, 6.143993..."
2,wood,"(((34.1773601, 59.0005708), (34.158443, 59.001...","((34.1354441, 59.0373975), (34.1272688, 59.030...","(9, 8, 4, 3, 1, 0, 14, 13, 12)","(3.816845040652781, 4.042617686739325, 4.08443..."
3,wood,"(((34.138577, 59.0398371), (34.1394997, 59.044...","((34.138577, 59.0398371), (34.1413021, 59.0442...","(0, 8, 6, 5, 4)","(1.0144503933649736, 1.692689120943144, 2.3324..."
4,wood,"(((34.1255882, 59.0993149), (34.1379478, 59.09...","((34.0558937, 59.086618), (34.06585, 59.077621...","(5, 4, 2, 1, 0, 8)","(5.548385512807122, 6.2804952476977265, 0.1261..."
5,wood,"(((34.1049888, 59.0525599), (34.1128853, 59.04...","((34.1273048, 59.0324268), (34.1355446, 59.071...","(3, 28, 27, 21, 15, 14, 13, 12, 11, 5, 4)","(1.3653886189193702, 1.563229980624813, 2.2950..."
6,wood,"(((34.0228613, 59.0155847), (34.0388017, 59.00...","((34.0020418, 59.0250026), (34.0096109, 59.003...","(11, 7, 6, 4, 15, 14, 12)","(5.04762648423328, 5.858518048190597, 6.143993..."
7,wood,"(((34.0342482, 59.0001777), (34.0554437, 59.00...","((34.0342482, 59.0001777), (34.0554437, 59.001...","[0, 1]",None
8,wood,"(((34.0004138, 59.0764101), (34.0141467, 59.07...","((34.0381793, 59.0635282), (34.0460757, 59.074...","(9, 7, 5, 4, 3, 0, 13, 10)","(0.9259921125969726, 2.0225289061735605, 2.238..."


Visibility graph can be built and (optionally) saved as networkx graph and (optionally) visualised using mplleaflet:

In [3]:
%%time
import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = vgraph.build_graph(inside_percent=0,
                            multiprocessing=False,
                            graph=True,
                            map_plot=map_plot)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  5.46it/s]


edges:  566
nodes:  185
Wall time: 3.14 s


c:\users\kozub\offroad-routing-engine\venv\lib\site-packages\IPython\core\display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


VisibilityGraph may also be used to find incident edges for a single point.  
This feature is used for pathfinding without graph building:

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet

start = ((34.02, 59.01), None, None, None, None)
incidents = vgraph.incident_vertices(start)

fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)